# Рекомендация тарифов

<span style="font-size: 20px;">Описание проекта</span> 

Оператор мобильной связи **«Мегалайн»** обнаружил, что многие клиенты продолжают использовать архивные тарифы.  
Задача — построить систему, которая проанализирует поведение пользователей и порекомендует им один из актуальных тарифов: **«Смарт»** или **«Ультра»**.

В распоряжении имеются данные о клиентах, которые уже перешли на один из этих тарифов.  
Необходимо построить **модель классификации**, способную на основе поведения пользователя определить, какой тариф ему подходит.

Предобработка данных уже выполнена.  
Качество модели оценивается по метрике **accuracy**. Необходимо, чтобы точность модели на тестовой выборке была не менее **0.75**.


<span style="font-size: 20px;">Описание данных</span> 

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `сais_ultralls` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Познакомимся с данными

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# познакомимся с таблицей
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
# посмотрим общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разобьем данные на выборки

In [5]:
# разделим признаки
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [6]:
# разобьем на обучающую и временную
features_train, features_temp, target_train, target_temp = train_test_split(
    features, target, test_size=0.25, random_state=12345)

# разобьем на валидационную и тестовую
features_valid, features_test, target_valid, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

## Рассмотрим и протестируем модели

### Дерево решений

In [7]:
# лучшая глубина для разных выборок 
best_depth_tree_valid = 0

# лучший accurancy для разных выборок 
best_accurancy_tree_valid = 0

# лучшая модель по валидационной выборке 
best_model_tree_valid = None

for depth in range(1, 26):
    
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    
    prediction = model.predict(features_valid)
    accurancy_valid = accuracy_score(prediction, target_valid)
    
    if accurancy_valid > best_accurancy_tree_valid:
        
        best_depth_tree_valid = depth
        best_accurancy_tree_valid = accurancy_valid
        best_model_tree_valid = model

print(f'Лучий результат на валидационной выборке: {best_accurancy_tree_valid}, \
при глубине: {best_depth_tree_valid}')

Лучий результат на валидационной выборке: 1.0, при глубине: 25


Как мы видим, результаты для разных выборок различаются, что может говорить о более сложных данных в тестовой выборке.

- На валидационной выборке модель демонстрирует отличные результаты, достигая 100% точности. 
- Однако на тестовой выборке, начиная с 7-го шага, наблюдается снижение точности модели, что может говорить о переобучении.

### Случайный лес

In [8]:
# лучшая глубина для разных выборок 
best_depth_forest_valid = 0

# лучший accurancy для разных выборок 
best_accurancy_forest_valid = 0

# лучшее число деревьев для разных выборок 
best_est_forest_valid = 0

# лучшая модель по валидационной выборке 
best_model_forest_valid = None

for n in range(10, 101, 10):
    
    for depth in range(1, 31):
    
        model = RandomForestClassifier(max_depth=depth, random_state=12345, n_estimators=n)
        model.fit(features_train, target_train)
    
        prediction = model.predict(features_valid)
        accurancy_valid = accuracy_score(prediction, target_valid)
        
        if accurancy_valid > best_accurancy_forest_valid:
            
            best_accurancy_forest_valid = accurancy_valid
            
            best_depth_forest_valid = depth
            best_est_forest_valid = n
            best_model_forest_valid = model
            
print(f'Лучий результат на валидационной выборке: {best_accurancy_forest_valid}, \
при числе деревьев: {best_est_forest_valid} и глубине: {best_depth_tree_valid}')

Лучий результат на валидационной выборке: 1.0, при числе деревьев: 60 и глубине: 25


С деревом решений ситуация аналогичная. Гиперпараметры, при которых достигаются наилучшие результаты, для разных выборок не совпадают:
- На валидационной выборке с увеличением числа деревьев и глубины точность достигает максимума при 60 деревьях и глубине 23.
- На тестовой выборке модель достигает пика точности при 20 деревьях и глубине 9, после чего точность начинает снижаться, что свидетельствует о переобучении.

### Логистическая регрессия

In [9]:
# лучший accurancy для разных выборок 
best_accurancy_line_valid = 0

# лучшее число повторений для разных выборок
max_iter_valid = 0

# лучшая модель по валидационной выборке 
best_model_line_valid = None

for i in range(100, 10001, 100):
    
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=i)
    model.fit(features_train, target_train)
    
    accurancy_valid = model.score(features_valid, target_valid)
    
    if accurancy_valid > best_accurancy_line_valid:
        
        best_accurancy_line_valid = accurancy_valid
        max_iter_valid = i
        best_model_line_valid = model

print(f'Лучий результат на валидационной выборке: {best_accurancy_line_valid}, \
при числе повторений: {max_iter_valid}')

Лучий результат на валидационной выборке: 0.7024896265560165, при числе повторений: 100


При логистической регрессии наилучшие результаты для обеих выборок были достигнуты при наименьшем числе итераций. Это может говорить о том, что модель быстро учится и дальнейшее увеличение числа итераций может приводить к переобучению.

#### Проверка лучшей модели

Проверим модель случайного леса, которая показала лучший результат на валидационной выборке, на тестовых данных.

In [10]:
best_model_forest_valid.score(features_test, target_test)

0.7935323383084577

На тестовых данных точность снизилась до **79%**, что всё еще соответствует целевому показателю. 

## Проверим модели на адекватность

In [11]:
# gосмотрим на распределение целевого признака по исходным данным
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

В исходном наборе данных значение 0 целевого признака, предсказываемого нашей моделью, встречается в 69.35% случаев. Если модель будет предсказывать всегда 0, то она будет права с вероятностью 69.35%. Значит, accuracy нашей модели должно быть выше этого показателя, чтобы считаться адекватной.

**Случайный лес**

Лучий результат на тестовой выборке: 0.7935323383084577, при числе деревьев: 60 и глубине: 23  
 
 - **accuracy** случайного леса на тестовой выборке **79%**, что выше 69.35% — модель адекватна.


## Вывод

Лучший результат на тестовой выборке показала модель **"Случайный лес"** с точностью **79%**, что соответствует:

- целевому показателю
- более высокой точности, чем наивное предсказание.

Она лучше всего подходит для предсказания целевого признака в этой задаче. 